In [15]:
%pip install pandas
%pip install prophet


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from prophet import Prophet
import pandas as pd

In [19]:
df_bills = pd.read_csv('data/bills.csv')
df_venues = pd.read_csv('data/venues.csv')

In [20]:
df_bills

,bill_paid_at_local,bill_total_billed,bill_total_discount_item_level,bill_total_gratuity,bill_total_net,bill_total_tax,bill_total_voided,bill_uuid,business_date,order_duration_seconds,order_seated_at_local,order_closed_at_local,order_take_out_type_label,order_uuid,payment_amount,payment_count,payment_total_tip,sales_revenue_with_tax,venue_xref_id,waiter_uuid
0,2024-07-01 09:17:01,21.81,0.00,0.0,19.30,2.51,0.0,240701091701~4D63608F-523C-4EFF-9A4F-78D6C44B5159,2024-07-01,46,2024-07-01 09:16:15,2024-07-01 09:17:01,dinein,240701091615~EF1C6E91-B6C4-4DF1-8A92-1B024197FEC3,21.81,1,0.0,21.81,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
1,2024-07-01 13:37:00,48.95,0.00,0.0,43.32,5.63,0.0,240701133659~3A0BEDE2-E9E5-484D-B909-780E485F0D69,2024-07-01,288,2024-07-01 13:32:12,2024-07-01 13:37:00,dinein,240701133212~A4C33BFA-A54F-4627-B0C2-7428427FB5DE,48.95,1,0.0,48.95,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
2,2024-07-01 11:19:32,26.94,0.00,0.0,23.84,3.10,0.0,240701111931~670EEB85-E939-4924-B92F-C95076B7E930,2024-07-01,135,2024-07-01 11:17:17,2024-07-01 11:19:32,dinein,240701111717~6074B0AB-2432-484D-BC3F-55CC5D731818,26.94,1,0.0,26.94,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
3,2024-07-01 15:05:59,3.96,0.00,0.0,3.77,0.19,0.0,240701150558~3D6F473E-D5EF-4BB8-AFAB-30DB2A0764C0,2024-07-01,48,2024-07-01 15:05:11,2024-07-01 15:05:59,dinein,240701150511~EAA68F41-20ED-4FC7-B902-B0B3878AEC82,3.96,1,0.0,3.96,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
4,2024-07-01 12:41:00,3.36,0.00,0.0,3.20,0.16,0.0,240701124059~0521794C-51FC-4C30-A368-995CE4DE105F,2024-07-01,103,2024-07-01 12:39:17,2024-07-01 12:41:00,dinein,240701123917~27764E1C-7E7F-4293-ABC7-5921310A654A,3.36,1,0.0,3.36,39e5b4830d4d9c14db7368a95b65d5463ea3d095203737...,161027134744~719D0E09-5CA7-442A-B9FA-BBBF4083B4FD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954781,2024-12-29 20:32:54,82.42,0.00,0.0,82.42,0.00,0.0,241229203254~464AA06D-F986-4C51-8A53-A7B8362072D6,2024-12-29,4382,2024-12-29 19:19:52,2024-12-29 20:32:54,dinein,241229191952~96F91DE2-5ECC-4AC9-9228-B0741F3494B1,82.42,1,0.0,82.42,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,220831171122~1455DBA8-211C-4281-BF27-8698365CA3EB
8954782,2024-12-30 20:36:19,87.92,0.00,0.0,87.92,0.00,0.0,241230203619~37348A86-98D1-4113-8EDD-06A3CF61A85B,2024-12-30,5244,2024-12-30 19:08:55,2024-12-30 20:36:19,dinein,241230190855~BB4604F2-B7BA-487A-99AF-7AEB77CA3EEC,87.92,1,0.0,87.92,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6
8954783,2024-12-31 14:24:23,51.96,0.00,0.0,51.96,0.00,0.0,241231142423~9D2E9295-9E59-4E76-8242-D230C9FD2494,2024-12-31,963,2024-12-31 14:08:20,2024-12-31 14:24:23,takeout,241231140820~4D531668-3D68-4789-A1DB-BE324D4583D5,51.96,1,0.0,51.96,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,161216233231~64B016BA-D9BA-49C7-8726-4BCB6516A11D
8954784,2024-12-31 17:00:34,49.35,11.59,0.0,49.35,0.00,0.0,241231170034~A9B4401D-26DC-4FC5-A4F9-0261D950DB3B,2024-12-31,59,2024-12-31 16:59:35,2024-12-31 17:00:34,takeout,241231165935~27989697-086C-4F7C-8AB2-8C0A8B2CF3A7,49.35,1,0.0,49.35,45f1736264898588301e2983fd2de6969a9af33aa0f6f8...,230522185246~9369CFD3-1EED-48B1-A116-F8A4F6B66FC6


In [21]:
df_venues

,venue_xref_id,concept,city,country,start_of_day_offset
0,ea74268311cfcc47d2b4c38ef08b9dab0117226a9a99a9...,FAST_FOOD,Albuquerque,US,00:00:00
1,e7cec41c46a9706ba1a702b368be5431913b6dc9c9da49...,FAMILY_DINING,Brooklyn,US,00:00:00
2,e63e79791883b0c4ac71d41eaee727932c13d6a4ec8f78...,FAST_CASUAL,Windsor,CA,00:00:00
3,c100388c9328b30b6bdfbea43f113d367ae9d32b03e84b...,BAR,Burnaby,CA,01:00:00
4,a819ee7f83f27dd3d7f44515e22aca3c26e6704ddd2a9b...,FAMILY_DINING,Tisdale,CA,00:00:00
...,...,...,...,...,...
596,cfd1fa6e8f122e93b1cb02e7b7541749335e7088095f8e...,NaN,Toronto,CA,00:00:00
597,75e7251298f2d62406dbfe1a11011cffdad90b27704687...,NaN,Richardson,US,00:00:00
598,40c38939612f59ad418569e94a075935576bff0449c7f8...,FAST_CASUAL,Guelph,CA,09:00:00
599,2ed5bf26505a6064d09b40bacdb6fccc9665c3de029494...,NaN,Mississauga,CA,00:00:00


# Data Preprocessing

# Machine Learning Model